In [1]:
import os
import numpy as np
import pandas as pd
import re
from scipy import stats

# from utils.logger import LOGPATH

In [5]:
# fpath = LOGPATH.joinpath('summary-fb.csv')
fpath = '/home/nliao/code/Spectral-GNN-Benchmark/log/summary-fb.csv'
t = pd.read_csv(fpath, engine='python')

fpath = '/home/nliao/code/Spectral-GNN-Benchmark/log/summary-fb-f.csv'
t2 = pd.read_csv(fpath, engine='python')

t_id = t.apply(lambda row: '+'.join((str(row['seed']), row['data'], row['model'], row['conv'])), axis=1)
t2_id = t2.apply(lambda row: '+'.join((str(row['seed']), row['data'], row['model'], row['conv'])), axis=1)
t2_id.drop_duplicates(keep='last', inplace=True)
t2 = t2.loc[t2_id.index]

row_id = []
for string in t2_id:
    match_index = t[t_id == string].index
    row_id.append(match_index[0])
t2.index = row_id
t.update(t2)

t['s_test'] = t.apply(lambda row: row[row['metric'] + '_test'], axis=1)
# t['zscore'] = t.groupby(['data', 'model', 'conv'])['s_test'].transform(lambda x: stats.zscore(x, ddof=0))
t['time_learn'] = t['time_learn'] * 1000 / t['epoch_learn']
t['time_eval'] = t['time_eval'] * 1000
t.drop(columns=['epoch_learn'], inplace=True)

t['median'] = t.groupby(['data', 'model', 'conv'])['s_test'].transform('median')
t['std'] = t.groupby(['data'])['s_test'].transform('std')
outlier = t['s_test'] < t['median'] - 2 * t['std']
t[outlier]

,seed,data,metric,model,conv,epoch_best,time_learn,mem_ram_train,mem_cuda_train,s_acc_train,...,s_acc_test,s_ap_test,s_auroc_test,s_f1i_test,time_eval,mem_ram_eval,mem_cuda_eval,s_test,median,std
110,20,cora,s_f1i,DecoupledFixed,AdjConv-gaussian,32,13.8312,1.221,0.046,51.412,...,51.885,53.622,88.612,67.837,3.4,1.221,0.046,67.837,86.0445,5.594013
375,25,citeseer,s_f1i,DecoupledVar,FavardConv,1,23.5926,1.226,0.118,16.667,...,16.667,16.409,49.012,20.305,2.1,1.226,0.118,20.305,74.5035,5.975701
500,21,pubmed,s_f1i,DecoupledVar,ClenshawConv,293,21.6354,1.227,0.274,55.887,...,55.021,58.135,77.515,48.605,4.0,1.227,0.274,48.605,84.6525,5.361021
503,23,pubmed,s_f1i,DecoupledVar,ClenshawConv,7,20.4166,1.223,0.274,33.339,...,33.367,32.849,49.294,27.270,3.8,1.223,0.274,27.270,84.6525,5.361021
508,28,pubmed,s_f1i,DecoupledVar,ClenshawConv,221,23.5978,1.225,0.274,57.902,...,57.404,74.592,85.848,68.848,4.2,1.225,0.274,68.848,84.6525,5.361021
658,27,squirrel_filtered,s_f1i,DecoupledFixed,AdjConv-mono,9,14.9420,1.301,0.050,28.616,...,20.093,26.996,59.989,26.126,1.6,1.301,0.050,26.126,33.8965,3.651296
778,28,flickr,s_f1i,DecoupledVar,ClenshawConv,1,25.2682,1.222,0.129,21.810,...,21.111,21.447,51.577,19.145,4.4,1.222,0.129,19.145,31.1515,4.204568
885,25,flickr,s_f1i,DecoupledVar,OptBasisConv,359,27.8440,1.262,0.227,23.870,...,21.177,20.144,49.683,18.947,5.1,1.262,0.227,18.947,33.8490,4.204568
895,20,chameleon_filtered,s_f1i,DecoupledVar,AdjiConv,64,22.4012,1.216,0.041,35.336,...,23.492,28.016,59.271,24.157,4.2,1.216,0.041,24.157,34.5505,4.827975
1192,20,squirrel_filtered,s_f1i,DecoupledVar,ChebConv,2,12.6904,1.236,0.063,20.000,...,20.000,20.000,50.000,31.306,1.0,1.236,0.063,31.306,38.9640,3.651296


In [6]:
dlst = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire", ]
        # "amazon_ratings", "minesweeper", "tolokers", "questions", "reddit", "penn94", \
        # "ogbn-arxiv", "arxiv-year", "genius", "twitch-gamer", "ogbn-mag"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['time_learn', 'mem_ram_train', 'mem_cuda_train', 's_test', 'time_eval',]
conv_repr = {
    'MLP': {'Identity': 'Identity',},
    'DecoupledFixed': {
        'AdjiConv-ones':    'Linear',
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',},
    'DecoupledVar': {
        'AdjiConv':         'Var-Linear',
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ClenshawConv':     'Clenshaw',
        'ChebIIConv':       'ChebInterp',
        'BernConv':         'Bernstein',
        'LegendreConv':     'Legendre',
        'JacobiConv':       'Jacobi',
        'FavardConv':       'Favard',
        'OptBasisConv':     'OptBasis',},
    'AdaGNN': {'LapiConv':  'AdaGNN',},
        # 'OptBasisConv':     'OptBasisGNN',},
    # 'ACMGNN': {'ACMConv-2.0-low-high-id':  'ACMGNN',}
        # 'ACMConv-1.0-low-high':     'FBGNNI',
        # 'ACMConv-2.0-low-high':     'FBGNNII',
        # 'ACMConv-1.0-low-high-id':  'ACMGNNI',
        # 'ACMConv-2.0-low-high-id':  'ACMGNNII',},
    'DecoupledFixedCompose': {
        'AdjiConv,AdjiConv-ones,ones': 'FAGNN',
        'Adji2Conv,Adji2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',},
    'DecoupledVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',}}

def reverse_name(name):
    for k, m in conv_repr.items():
        for c, n in m.items():
            if n == name:
                return k, c
    return None, None

flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

def get_df(t):
    tt = t[t['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
    # print(tt.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 10))
    df = tt.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace=True)

    df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
    df.dropna(subset=['name'], inplace=True)
    df.drop(columns=['model', 'conv'], inplace=True)
    df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
    return df

t = t[~outlier]
df = get_df(t)
df

,data,name,time_learn_mean,time_learn_std,mem_ram_train_mean,mem_ram_train_std,mem_cuda_train_mean,mem_cuda_train_std,s_test_mean,s_test_std,time_eval_mean,time_eval_std
0,actor,AdaGNN,15.26536,1.587502,1.3451,0.045489,0.1150,0.000000,36.4934,1.208714,1.34,0.241293
1,actor,PPR,15.30964,0.550513,1.3223,0.035352,0.0819,0.000316,35.1712,1.014586,1.35,0.108012
2,actor,Gaussian,14.73958,0.907781,1.3174,0.035189,0.0770,0.000000,36.4408,1.254801,1.40,0.188562
3,actor,HK,13.51230,2.048980,1.3218,0.039830,0.0819,0.000316,37.5329,1.248192,1.21,0.119722
4,actor,Impulse,14.45874,0.536065,1.3399,0.039948,0.0770,0.000000,27.0001,1.302628,1.33,0.048305
...,...,...,...,...,...,...,...,...,...,...,...,...
179,squirrel_filtered,Jacobi,17.22644,1.255399,1.2739,0.024452,0.0620,0.000000,36.8694,1.599719,2.27,0.326769
180,squirrel_filtered,Legendre,15.17182,1.670774,1.2800,0.020629,0.0620,0.000000,35.5179,1.501591,1.83,0.365300
181,squirrel_filtered,OptBasis,24.50418,2.631535,1.3229,0.024628,0.0920,0.000000,33.7613,1.912644,12.97,26.095723
182,squirrel_filtered,FiGURe,37.87520,2.627503,1.3028,0.023673,0.0880,0.000000,36.9369,2.355155,6.04,0.836926


## Fine

In [7]:
# dlst_acc = dlst
dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
# dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "reddit", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
# dlst_acc = ["ogbn-arxiv", "amazon_ratings", "minesweeper", "tolokers", "questions", "penn94", "arxiv-year", "genius", "twitch-gamer"]

acc_dual = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst_acc, ['mean', 'std']]), dtype=float)
acc_str = pd.DataFrame(index=flst, columns=dlst_acc, dtype=str)
for row in acc_str.index:
    for col in acc_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == col), 's_test_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == col), 's_test_std'].values
        if len(mean) > 0:
            acc_dual.loc[row, col] = mean[0], std[0]
            acc_str.loc[row, col] = f'{mean[0]:.2f}\\tpm{{{std[0]:.2f}}}'

acc_dual.style.background_gradient(axis=0)

In [51]:
def form_sh(model, conv, data, seed):
    s = f'{seed:.0f} --data {data} '
    if model in ['DecoupledFixed', 'DecoupledFixedCompose']:
        theta_scheme = conv.split('-')[1]
        s += f'--model {model} --conv {conv.split('-')[0]} --theta_scheme {theta_scheme}'
    else:
        s += f'--model {model} --conv {conv}'

    file_dct = {
        "DecoupledFixed": "ffb_fix.txt",
        "DecoupledVar": "ffb_var.txt",
        "DecoupledFixedCompose": "ffb_bank.txt",
        "DecoupledVarCompose": "ffb_bank.txt",
    }
    with open(file_dct[model], 'a') as f:
        f.write(s + '\n')
    return s

acc_mean = acc_dual.xs('mean', level=1, axis=1)

acc_std = acc_dual.xs('std', level=1, axis=1)
std_agg = acc_std.agg(['median', 'std'], axis=0)

df_out = acc_std[acc_std > (std_agg.loc['median'] + np.sqrt(std_agg.loc['median'])*std_agg.loc['std'])]
for nme, data in df_out.stack().index.tolist():
    model, conv = reverse_name(nme)
    if nme in ['Identity', 'Var-Linear']:
        continue
    acc_all_mean = acc_dual.xs('mean', level=1, axis=1).mean(axis=0).loc[data]
    acc_mean = acc_dual.loc[nme, (data, 'mean')] - np.sqrt(acc_std[data].mean())
    # acc_mean = min(acc_mean, acc_all_mean)
    condition_below = t.loc[(t['data'] == data) & (t['model'] == model) & (t['conv'] == conv), 's_test'] < acc_mean
    seeds = t.loc[(t['data'] == data) & (t['model'] == model) & (t['conv'] == conv) & condition_below, 'seed'].values
    for seed in seeds:
        print(model, conv, data, seed)
        # form_sh(model, conv, data, seed)

## Latex - Acc

In [8]:
def calc_ranks(mean, std, ascending=False, exc=[]):
    rank = pd.Series(np.nan, index=mean.index)
    na_indices = mean[mean.isna()].index
    nrows = len(mean)
    rank[na_indices] = nrows

    mean = mean.dropna()
    mean = mean[~mean.index.isin(exc)]
    std = std.loc[mean.index]
    idx_pend = mean.sort_values(ascending=ascending).index
    current_rank = 1
    while not idx_pend.empty and current_rank <= nrows:
        idx_top = idx_pend[0]
        top_mean = mean[idx_top]
        top_std = std[idx_top]
        rank[idx_top] = current_rank

        if ascending:
            idx_same = idx_pend[mean[idx_pend] <= top_mean + top_std]
        else:
            idx_same = idx_pend[mean[idx_pend] >= top_mean - top_std]
        rank[idx_same] = current_rank

        idx_pend = idx_pend.difference(idx_same)
        idx_pend = mean.loc[idx_pend].sort_values(ascending=ascending).index
        current_rank += len(idx_same)

    return rank

rk = ['a', 'b', 'c']

In [10]:
# acc_str[acc_str.isna().any(axis=1)]
# acc_str.dropna(inplace=True)
acc_str.fillna('(OOM)', inplace=True)

for col in acc_str.columns:
    mean = df.loc[df['data'] == col, ['name', 's_test_mean']].set_index('name')['s_test_mean'].astype(float)
    std  = df.loc[df['data'] == col, ['name', 's_test_std']].set_index('name')['s_test_std'].astype(float)
    # rank = calc_ranks(mean, std)
    rank = mean.rank(ascending=False).astype(int)
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        acc_str.loc[idx, col] = f'\\rk{rki}' + '{' + acc_str.loc[idx, col] + '}'

acc_str

,cora,citeseer,pubmed,flickr,chameleon_filtered,squirrel_filtered,actor,roman_empire
Identity,75.60\tpm{1.08},72.92\tpm{1.34},87.31\tpm{0.48},35.95\tpm{1.27},32.08\tpm{3.86},24.37\tpm{5.59},35.95\tpm{1.27},66.23\tpm{0.74}
Linear,86.62\tpm{1.69},76.00\tpm{1.23},84.52\tpm{0.49},26.22\tpm{1.09},33.93\tpm{3.30},36.62\tpm{2.64},25.08\tpm{0.46},30.92\tpm{0.82}
Impulse,86.21\tpm{1.53},74.79\tpm{1.03},83.02\tpm{0.74},26.48\tpm{0.83},34.21\tpm{3.71},35.23\tpm{2.20},27.00\tpm{1.30},27.90\tpm{0.59}
Monomial,80.91\tpm{3.37},76.26\tpm{1.67},89.22\tpm{0.93},32.36\tpm{2.81},37.64\tpm{3.56},33.91\tpm{1.31},35.53\tpm{1.40},60.71\tpm{1.37}
PPR,\rka{88.61\tpm{0.95}},\rka{77.25\tpm{1.29}},89.01\tpm{0.38},35.55\tpm{1.04},32.81\tpm{3.57},33.58\tpm{1.59},35.17\tpm{1.01},62.26\tpm{2.34}
HK,\rkb{88.06\tpm{0.99}},\rkc{77.10\tpm{0.89}},89.11\tpm{0.63},\rka{37.33\tpm{1.48}},32.64\tpm{4.06},34.28\tpm{1.50},\rkb{37.53\tpm{1.25}},65.60\tpm{0.92}
Gaussian,85.11\tpm{4.26},76.92\tpm{1.16},89.06\tpm{0.76},35.62\tpm{0.87},33.54\tpm{4.02},37.18\tpm{1.82},36.44\tpm{1.25},\rkc{66.47\tpm{0.94}}
Var-Linear,87.08\tpm{1.46},76.00\tpm{1.14},85.04\tpm{0.75},27.49\tpm{1.12},36.39\tpm{4.25},\rkc{37.32\tpm{1.85}},25.56\tpm{1.22},40.05\tpm{6.02}
Var-Monomial,87.56\tpm{1.00},76.72\tpm{1.25},88.18\tpm{0.76},35.42\tpm{1.89},\rkb{39.33\tpm{2.90}},\rka{40.65\tpm{1.67}},32.13\tpm{3.47},64.76\tpm{0.66}
Horner,86.14\tpm{1.82},76.72\tpm{1.61},89.59\tpm{0.45},\rkc{36.74\tpm{0.88}},37.92\tpm{3.41},34.57\tpm{1.35},\rka{37.65\tpm{1.10}},61.52\tpm{0.88}


In [11]:
for row in acc_str.index:
    print("\t& " + row + " & " + " & ".join(acc_str.loc[row].values) + " \\\\")
print()

	& Identity & 75.60\tpm{1.08} & 72.92\tpm{1.34} & 87.31\tpm{0.48} & 35.95\tpm{1.27} & 32.08\tpm{3.86} & 24.37\tpm{5.59} & 35.95\tpm{1.27} & 66.23\tpm{0.74} \\
	& Linear & 86.62\tpm{1.69} & 76.00\tpm{1.23} & 84.52\tpm{0.49} & 26.22\tpm{1.09} & 33.93\tpm{3.30} & 36.62\tpm{2.64} & 25.08\tpm{0.46} & 30.92\tpm{0.82} \\
	& Impulse & 86.21\tpm{1.53} & 74.79\tpm{1.03} & 83.02\tpm{0.74} & 26.48\tpm{0.83} & 34.21\tpm{3.71} & 35.23\tpm{2.20} & 27.00\tpm{1.30} & 27.90\tpm{0.59} \\
	& Monomial & 80.91\tpm{3.37} & 76.26\tpm{1.67} & 89.22\tpm{0.93} & 32.36\tpm{2.81} & 37.64\tpm{3.56} & 33.91\tpm{1.31} & 35.53\tpm{1.40} & 60.71\tpm{1.37} \\
	& PPR & \rka{88.61\tpm{0.95}} & \rka{77.25\tpm{1.29}} & 89.01\tpm{0.38} & 35.55\tpm{1.04} & 32.81\tpm{3.57} & 33.58\tpm{1.59} & 35.17\tpm{1.01} & 62.26\tpm{2.34} \\
	& HK & \rkb{88.06\tpm{0.99}} & \rkc{77.10\tpm{0.89}} & 89.11\tpm{0.63} & \rka{37.33\tpm{1.48}} & 32.64\tpm{4.06} & 34.28\tpm{1.50} & \rkb{37.53\tpm{1.25}} & 65.60\tpm{0.92} \\
	& Gaussian & 85.11\tpm{

## Latex - Eff

In [34]:
mlst = ["time_learn", "time_eval", "mem_cuda_train"]
dlst_eff = ["ogbn-arxiv", "amazon_ratings", "minesweeper", "tolokers", "questions", "penn94", "arxiv-year", "genius", "twitch-gamer"]
eff_str = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst_eff, mlst]))

for row in eff_str.index:
    for data, met in eff_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == data), met+'_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == data), met+'_std'].values
        if len(mean) > 0:
            eff_str.loc[row, (data, met)] = f"{mean[0]:.2f}"
        else:
            eff_str.loc[row, (data, met)] = "-"

for data,met in eff_str.columns:
    mean = df.loc[df['data'] == data, ['name', met+'_mean']].set_index('name')[met+'_mean'].astype(float)
    std  = df.loc[df['data'] == data, ['name', met+'_std']].set_index('name')[met+'_std'].astype(float)
    rko = ['Identity']
    rank = calc_ranks(mean, std, True, rko)
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        eff_str.loc[idx, (data, met)] = f'\\rk{rki}' + '{' + eff_str.loc[idx, (data, met)] + '}'
    eff_str.loc[rko, (data, met)] = '\\rko{' + eff_str.loc[rko, (data, met)].astype(str) + '}'

eff_str

ogbn-arxiv                            amazon_ratings  \
                time_learn   time_eval mem_cuda_train     time_learn   
Identity       \rko{10.37}  \rko{2.37}     \rko{0.56}     \rko{4.02}   
Impulse       \rka{103.47}  \rkb{4.70}     \rka{1.45}    \rka{12.00}   
Monomial      \rka{103.56}  \rkb{4.67}     \rka{1.45}    \rka{11.52}   
PPR           \rka{104.03}  \rkb{4.57}           1.50    \rka{11.99}   
HK            \rka{104.03}  \rkb{4.63}     \rka{1.45}    \rka{11.12}   
Gaussian            106.44  \rka{3.60}           1.57    \rka{11.69}   
Var-Monomial  \rka{104.69}  \rkb{4.53}           1.53          13.14   
Horner              117.04        9.77           2.54          17.66   
Chebyshev           118.37        7.00           2.48          15.70   
ChebInterp          249.60       13.33           2.56          45.87   
Clenshaw            123.03       10.73           2.54          17.45   
Bernstein           604.27       15.10          11.33          55.54   
AdaGNN              109.75        5.13           2.32          14.44   
ACMGNN              272.45       10.57           8.22          48.15   
FAGNN               224.73        6.40           2.47          21.44   
G$^2$CN             417.21        7.37           4.25          35.11   
GNN-LF/HF           216.75        7.47           2.39          21.21   
FiGURe              804.51      224.97          14.15          75.99   

                                        minesweeper              \
               time_eval mem_cuda_train  time_learn   time_eval   
Identity      \rko{0.90}     \rko{0.10}  \rko{3.44}  \rko{0.60}   
Impulse             2.83     \rkb{0.24}  \rka{8.02}        1.83   
Monomial      \rka{2.50}     \rkb{0.24}  \rka{8.02}  \rka{1.57}   
PPR           \rkc{2.70}     \rkb{0.24}  \rka{7.19}  \rka{1.53}   
HK            \rka{2.37}     \rkb{0.24}  \rka{7.51}  \rka{1.63}   
Gaussian            6.43     \rka{0.24}  \rka{7.95}  \rka{1.63}   
Var-Monomial  \rkc{2.63}     \rkb{0.24}        9.70  \rka{1.57}   
Horner              5.87           0.39       12.60        3.50   
Chebyshev           5.47           0.38       14.16        3.07   
ChebInterp          9.07           0.39       42.67        9.03   
Clenshaw            6.47           0.38       13.58        3.60   
Bernstein          12.17           1.65       32.37        8.67   
AdaGNN              4.67           0.35        9.94        2.00   
ACMGNN             13.47           1.22       38.95        7.73   
FAGNN               5.43           0.39       13.83        3.53   
G$^2$CN             7.30           0.64       17.09        3.80   
GNN-LF/HF           5.60           0.37       12.78        2.83   
FiGURe             25.30           2.07       42.86       12.40   

                                tolokers  ...         penn94   arxiv-year  \
             mem_cuda_train   time_learn  ... mem_cuda_train   time_learn   
Identity         \rko{0.04}   \rko{3.60}  ...     \rko{0.93}   \rko{9.03}   
Impulse          \rkc{0.10}  \rka{21.71}  ...     \rka{1.19}        98.09   
Monomial         \rka{0.10}  \rka{21.94}  ...     \rka{1.19}        95.60   
PPR              \rkc{0.10}  \rka{22.81}  ...     \rka{1.19}  \rka{85.22}   
HK               \rkc{0.10}  \rka{21.93}  ...     \rka{1.19}  \rkc{90.97}   
Gaussian         \rkb{0.10}  \rka{22.47}  ...           1.26        92.50   
Var-Monomial     \rkc{0.10}        23.75  ...     \rka{1.19}  \rka{85.26}   
Horner                 0.15        27.34  ...           1.49        99.48   
Chebyshev              0.15        26.14  ...           1.48        99.83   
ChebInterp             0.16        59.36  ...           1.49       171.22   
Clenshaw               0.15        25.75  ...           1.49       103.50   
Bernstein              0.69       128.71  ...           3.70       499.11   
AdaGNN                 0.15        23.85  ...           1.44  \rkc{90.89}   
ACMGNN                 0.51        64.56  ...           2.88       239.93   
FA

In [40]:
# dlst_eff = ["flickr", "reddit", "tolokers", "penn94"]
dlst_eff = ["penn94", "ogbn-arxiv", "genius", "twitch-gamer"]
for row in eff_str.index:
    values = eff_str.loc[row, dlst_eff].values.tolist()
    while "-" in values:
        idx = values.index("-")
        values[idx] = '\\multicolumn{3}{c|}{(OOM)}'
        for i in range(1, len(mlst)):
            values.pop(idx+1)
    print("\t& " + row + " & " + " & ".join(values) + " \\\\")
print()

	& Identity & \rko{9.03} & \rko{1.93} & \rko{0.93} & \rko{10.37} & \rko{2.37} & \rko{0.56} & \rko{12.21} & \rko{3.03} & \rko{0.95} & \rko{7.97} & \rko{1.97} & \rko{0.56} \\
	& Impulse & \rka{57.11} & \rkb{6.87} & \rka{1.19} & \rka{103.47} & \rkb{4.70} & \rka{1.45} & \rka{115.16} & \rka{6.73} & \rka{3.18} & \rka{149.00} & 4.90 & \rka{1.54} \\
	& Monomial & \rka{56.81} & \rkb{6.83} & \rka{1.19} & \rka{103.56} & \rkb{4.67} & \rka{1.45} & \rka{116.12} & \rka{6.73} & 3.41 & \rka{148.81} & 5.77 & \rka{1.54} \\
	& PPR & \rka{57.21} & \rkb{7.13} & \rka{1.19} & \rka{104.03} & \rkb{4.57} & 1.50 & 116.19 & \rka{6.77} & 3.41 & \rka{148.13} & \rkb{4.17} & \rka{1.54} \\
	& HK & \rka{57.62} & \rkb{6.87} & \rka{1.19} & \rka{104.03} & \rkb{4.63} & \rka{1.45} & \rka{115.53} & 7.10 & \rka{3.18} & \rka{148.15} & \rkb{4.13} & \rka{1.54} \\
	& Gaussian & \rka{56.88} & \rka{6.10} & 1.26 & 106.44 & \rka{3.60} & 1.57 & 116.31 & \rka{6.60} & 3.46 & \rka{148.10} & \rka{3.67} & 1.72 \\
	& Var-Monomial & \rka{57.8

## Latex - Full